# Balanced network of excitatory and inhibitory neurons: model change


Our aim was to change the structure and the parameters of the standard Vogels and Abbott (VA) Network, and to understand how these changes impacted the results and behavior of the model.

(NOTE: baseline code for functions etc. was taken from the source notebooks seen in the labs, contained in this GitHub repository)

In [1]:
fileName='VA_balance-network'

In [2]:
from pyNN import space
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pickle
import time
import datetime
import json
import warnings
warnings.filterwarnings('ignore')

savePath = f'./outputs/' # remember to create the folder if not already present (mkdir ./notebooks/outputs)
dt_string = datetime.datetime.today().isoformat() # ISO8601 ! :-)
tag = dt_string
saveName = f'{savePath}{fileName}-{tag}'
print(saveName)
PARS={}

./outputs/VA_balance-network-2024-02-08T11:28:50.490992


# step0: import the simulator

In [3]:
import socket
try:
    import pyNN.spiNNaker as sim
except ModuleNotFoundError:
    import pyNN.brian2 as sim
    
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time
from pyNN import space 

# step1: setup the simulator

In [4]:
dt = 1          # (ms) simulation timestep
tstop = 1000      # (ms) simulaton duration
delay = 2 # (ms) 

sim.setup(
    timestep=dt, 
    min_delay=delay, 
    max_delay=delay) # [ms] # not that the max_delay supported by SpiNNaker is timestep * 144

rngseed = 98766987
rng = NumpyRNG(seed=rngseed, parallel_safe=True)

2024-02-08 11:28:51 INFO: Read cfg files: /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2024-02-08 11:28:51 INFO: Will search these locations for binaries: /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/common_model_binaries : /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/model_binaries
2024-02-08 11:28:51 INFO: Setting time scale factor to 1.
2024-02-08 11:28:51 INFO: Setting machine time step to 1000 micro-seconds.


['/home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


# step2: decide the cell types and parameters

In [5]:
sim.list_standard_models()

['AbstractPyNNModel',
 'IF_cond_exp',
 'IF_curr_exp',
 'IF_curr_alpha',
 'IF_curr_delta',
 'Izhikevich',
 'SpikeSourceArray',
 'SpikeSourcePoisson']

In [6]:
# cell type
celltype = sim.Izhikevich
sim.Izhikevich.get_parameter_names()



dict_keys(['a', 'b', 'c', 'd', 'i_offset', 'tau_syn_E', 'tau_syn_I'])

In [7]:
standard_pars = sim.Izhikevich.default_parameters

standard_pars


{'a': 0.02,
 'b': 0.2,
 'c': -65.0,
 'd': 2.0,
 'i_offset': 0.0,
 'tau_syn_E': 5.0,
 'tau_syn_I': 5.0}

In [10]:
cell_params = standard_pars
cell_params['i_offset'] = 5.0
cell_params

#define initial values(default ones used)
v= -70.0
u = -14.0
#v=membrane potential.
#u=membrane recovery variable.
#i_offset=input current.
#a: Time scale of the recovery variable u.
#b: Sensitivity of the recovery variable u to the subthreshold fluctuations of the membrane potential v.
#c: Reset value of the membrane potential v caused by the fast high-threshold potassium conductance.
#d: Reset value of the recovery variable u caused by the slow high-threshold potassium conductance.

# step3: making cell populations


In [11]:
n = 1500          # number of cells
r_ei = 4.0        # number of excitatory cells:number of inhibitory cells
n_exc = int(round((n * r_ei / (1 + r_ei))))  # number of excitatory cells
n_inh = n - n_exc                            # number of inhibitory cells
print("{} {}".format(n_exc, n_inh))

pops = {
    'exc': sim.Population(
                                n_exc, 
                                celltype(**cell_params), 
                                label="Excitatory_Cells"),

    'inh': sim.Population(
                                n_inh, 
                                celltype(**cell_params), 
                                label="Inhibitory_Cells")
}

pops['exc'].record(['v', 'spikes'])
pops['inh'].record(['v', 'spikes'])

c = cell_params['c']
uniformDistr = RandomDistribution('uniform', [c, u], rng=rng)
pops['exc'].initialize(v=uniformDistr)
pops['inh'].initialize(v=uniformDistr)

pops.keys()

1200 300


dict_keys(['exc', 'inh'])

# step4: define the synapse types

In [12]:
# Synapse parameters
###UNCOMMENT THIS PART TO USE THE ORIGINAL WEIGHTS
#Gexc = None
#Ginh = None
#Gexc = 4.     # (nS) #1
#Ginh = 51.    # (nS) #1
#w_exc = Gexc * 1e-3             
#w_inh = Ginh * 1e-3

###COMMENT THIS PART TO USE THE ORIGINAL WEIGHTS
i_offset = cell_params['i_offset'] 
w_exc = i_offset * 0.2            
w_inh = i_offset * 0.5

exc_synapses = sim.StaticSynapse(weight=w_exc, delay=delay)
inh_synapses = sim.StaticSynapse(weight=w_inh, delay=delay)


# step5: select the connections algorithm

In [13]:
pconn = 0.02      # connection probability (2%)

exc_conn = sim.FixedProbabilityConnector(pconn, rng=rng)
inh_conn = sim.FixedProbabilityConnector(pconn, rng=rng)

# step6: make the projections

In [14]:
connections = {
    
    'e2e': sim.Projection(
        pops['exc'],
        pops['exc'], 
        exc_conn, 
        receptor_type='excitatory',
        synapse_type=exc_synapses),
    
    'e2i': sim.Projection(
        pops['exc'], 
        pops['inh'], 
        exc_conn, 
        receptor_type='excitatory',
        synapse_type=exc_synapses),
    
    'i2e': sim.Projection(
        pops['inh'], 
        pops['exc'], 
        inh_conn, 
        receptor_type='inhibitory',
        synapse_type=inh_synapses),
    
    'i2i': sim.Projection(
        pops['inh'],
        pops['inh'],
        inh_conn, 
        receptor_type='inhibitory',
        synapse_type=inh_synapses)
    
        }

connections.keys()

dict_keys(['e2e', 'e2i', 'i2e', 'i2i'])

# step7: setting the thalamic stimulus

In [15]:
n_thalamic_cells = 20 
stim_dur = 50.    # (ms) duration of random stimulation
rate = 100.       # (Hz) frequency of the random stimulation

exc_conn = None

pops['thalamus'] = sim.Population(
    n_thalamic_cells, 
    sim.SpikeSourcePoisson(rate=rate, duration=stim_dur),
    label="expoisson")
pops['thalamus'].record("spikes")


rconn = 0.01
ext_conn = sim.FixedProbabilityConnector(rconn)

connections['ext2e'] = sim.Projection(
    pops['thalamus'], 
    pops['exc'], 
    ext_conn, 
    receptor_type='excitatory',
    synapse_type=sim.StaticSynapse(weight=0.1))

connections['ext2i'] = sim.Projection(
    pops['thalamus'], 
    pops['inh'], 
    ext_conn, 
    receptor_type='excitatory',
    synapse_type=sim.StaticSynapse(weight=0.1))

connections.keys(), pops.keys()

(dict_keys(['e2e', 'e2i', 'i2e', 'i2i', 'ext2e', 'ext2i']),
 dict_keys(['exc', 'inh', 'thalamus']))

# step8: run simulation

In [16]:
tstop = 1000 # (ms)
# simulation run

tic = time.time()
sim.run(tstop)
toc = time.time() - tic

2024-02-08 11:32:24 INFO: Starting execution process
2024-02-08 11:32:24 INFO: Simulating for 1000 1.0ms timesteps using a hardware timestep of 1000us
Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2024-02-08 11:32:25 INFO: Time 0:00:00.016836 taken by SpynnakerSplitterSelector
Adding delay extensions as required
|0%                          50%                         100%|
2024-02-08 11:32:25 INFO: Time 0:00:00.015102 taken by DelaySupportAdder
2024-02-08 11:32:28 INFO: Time 0:00:03.033845 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2024-02-08 11:32:40 INFO: Time 0:00:12.192957 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2024-02-

# step9: save results

In [17]:
stateVars = {}
for pop in pops.keys():
    for recording in ['v', 'spikes']:
        pops[pop].write_data(f'{saveName}-{recording}.pkl')
        stateVars[pop]=pops[pop].get_data()



Getting spikes for Excitatory_Cells1
|0%                          50%                         100%|
Getting v for Excitatory_Cells1
|0%                          50%                         100%|
Getting spikes for Excitatory_Cells1
|0%                          50%                         100%|
Getting v for Excitatory_Cells1
|0%                          50%                         100%|
Getting spikes for Excitatory_Cells1
|0%                          50%                         100%|
Getting v for Excitatory_Cells1
|0%                          50%                         100%|
Getting spikes for Excitatory_Cells1
|0%                          50%                         100%|
Getting v for Excitatory_Cells1
|0%                          50%                         100%|
Getting spikes for Inhibitory_Cells2
|0%                          50%                         100%|
Getting v for Inhibitory_Cells2
|0%                          50%                         100%|
Getting spikes for Inhibi

# step10: recover results

In [18]:
def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        print(key)
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        VAR=outputs[key].segments[0].spiketrains
        results[key, 'spikes']=[]
        for k in range(len(VAR)):
            results[key, 'spikes'].append(np.array(list(VAR[k])).T,)
    return results
stateVars.keys()
results = recover_results(stateVars)
results.keys()

exc
v
inh
v
thalamus


dict_keys([('exc', 'v'), ('exc', 'spikes'), ('inh', 'v'), ('inh', 'spikes'), ('thalamus', 'spikes')])

# step11: postprocessing (looking at the results)

In [19]:
from pyNN.utility.plotting import Figure, Panel

In [20]:
import os
import matplotlib.pyplot as plt

outputs = 'outputs'

raster_plot_exc= Figure(
    # raster plot of the presynaptic neuron spike times
    Panel(stateVars['exc'].segments[0].spiketrains, xlabel="Time/ms", xticks=True,
          yticks=True, markersize=0.2, xlim=(0, tstop)),
    title="Vogels-Abbott benchmark: excitatory cells spikes")

# Save the graph image
output_file = os.path.join(outputs, 'raster_plot_exc')
plt.savefig(output_file)

print(f"Graph saved as {output_file}")

Canvas(toolbar=None)

Graph saved as outputs/raster_plot_exc


In [21]:
raster_plot_inh= Figure(
    # raster plot of the presynaptic neuron spike times
    Panel(stateVars['inh'].segments[0].spiketrains, xlabel="Time/ms", xticks=True,
          yticks=True, markersize=0.2, xlim=(0, tstop)),
    title="Vogels-Abbott benchmark: inhibitory cells spikes")

output_file_2 = os.path.join(outputs, 'raster_plot_inh')
plt.savefig(output_file_2)

print(f"Graph saved as {output_file_2}")

Graph saved as outputs/raster_plot_inh


In [22]:
# check membrane voltage potentials
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1,figsize=(6, 8), sharey=True, sharex=False)
ax.hist(np.asarray(results['exc', 'v']).ravel(), 100, label='exc', log=False)
ax.hist(np.asarray(results['inh', 'v']).ravel(), 100, label='inh')

ax.set_xlim(-90,-40)

ax.legend()
ax.set_ylabel('excitatory cells', fontsize=14)
ax.set_xlabel('membrane potential [mV]', fontsize=14) 

plt.savefig('membrane potential')


Canvas(toolbar=None)

In [23]:
fig, ax = plt.subplots(2,1, sharex=True, figsize=(9,5))
p0=ax[0].eventplot(results['exc', 'spikes'], color='green')
p1=ax[1].eventplot(results['inh', 'spikes'], color='red')
[ax[i].set_xlabel('Time (ms)', fontsize=14) for i in [0,1]]
[ax[i].set_ylabel(feat, fontsize=14) for i, feat in enumerate(['Excitatory cells', 'Inhibitory cells'])]

plt.savefig('inh_exc')

Canvas(toolbar=None)

# step12: close simulation


In [ ]:
# if you run it, you cannot reload some cells, but it is needed to restart the overall simulation

#sim.end()